In [145]:
import pandas as pd
import re
import datetime

from disaster_extr_constants import *
from disaster_extr_helpers import *

In [253]:
YEAR = 2015
disaster_type = 'storm'
disaster_YEAR_neg = storm_tags_2015_pos
disaster_YEAR_pos = storm_tags_2015_neg
file_path = 'data/'+str(YEAR)+'_'+disaster_type+'_climate_processed_csv.bz2'

In [254]:
data = 'data/emdat_processed.csv'
parse_dates = ['StartDate', 'EndDate']
df_emdat = pd.read_csv(data, index_col="Dis No", parse_dates = parse_dates)

In [255]:
disaster_df_quotes = pd.read_csv(file_path, parse_dates = ['date'], compression='bz2')

In [256]:
if disaster_type == 'heat_wave':
    
    if YEAR == 2020:
        df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES_2020, HEAT_WAVES_2020_val)
        df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)
    else:
        df_heat_wave = get_df_disaster(df_emdat, HEAT_WAVES, HEAT_WAVES_val)
        df_heat_wave_bounds = retrieve_bounding_dates(df_heat_wave)
        
    disaster_df = df_heat_wave
    disaster_df_bounds = df_heat_wave_bounds
        
elif disaster_type == 'storm':
    
    if YEAR == 2020:
        df_storm = get_df_disaster(df_emdat, STORMS_2020, STORMS_2020_val)
        df_storm_bounds = retrieve_bounding_dates(df_storm)
    else:
        df_storm = get_df_disaster(df_emdat, STORMS, STORMS_val)
        df_storm_bounds = retrieve_bounding_dates(df_storm)

    disaster_df = df_storm
    disaster_df_bounds = df_storm_bounds
    
if YEAR == 2020:
    lookupYEAR = YEAR-1
else:
    lookupYEAR = YEAR
    

start_YEAR, end_YEAR = disaster_df_bounds.loc[lookupYEAR].MinStartDate, disaster_df_bounds.loc[lookupYEAR].MaxEndDate

In [257]:
disaster_df[['Type', 'Subtype','Name', 'Country','StartDate','EndDate']]

,Type,Subtype,Name,Country,StartDate,EndDate
Dis No,,,,,,
2015-0470-MEX,Storm,Tropical cyclone,Hurricane Patricia,Mexico,2015-10-22,2015-10-28
2016-0041-FJI,Storm,Tropical cyclone,Cyclone Winston,Fiji,2016-02-20,2016-02-21
2017-0362-USA,Storm,Tropical cyclone,Hurricane Harvey,United States of America (the),2017-08-25,2017-08-29
2018-0341-CHN,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),China,2018-09-10,2018-09-18
2018-0342-USA,Storm,Tropical cyclone,Hurricane Florence,United States of America (the),2018-09-12,2018-09-18
2018-0341-PHL,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),Philippines (the),2018-09-16,2018-09-16
2018-0341-HKG,Storm,Tropical cyclone,Typhoon Mangkut (Ompong),Hong Kong,2018-09-17,2018-09-17
2019-0492-JPN,Storm,Tropical cyclone,Tropical cylone 'Hagibis',Japan,2019-10-12,2019-10-17


In [258]:
disaster_df_bounds

,MinStartDate,MaxEndDate
Year,,
2015,2015-10-22,2015-10-28
2016,2016-02-20,2016-02-21
2017,2017-08-25,2017-08-29
2018,2018-09-10,2018-09-18
2019,2019-10-12,2019-10-17


In [259]:
print("Disaster [{}] Dates: {} --- {}".format(
        disaster_type,
        start_YEAR.strftime("%Y-%m-%d"), 
        end_YEAR.strftime("%Y-%m-%d")))

Disaster [storm] Dates: 2015-10-22 --- 2015-10-28


In [260]:
one_month = datetime.timedelta(days=31)

if end_YEAR - start_YEAR < one_month:
    # Add 10 days to end date
    ten_days = datetime.timedelta(days=10)
    end_YEAR += ten_days

In [261]:
print("Used extraction dates: {} --- {}".format(
        start_YEAR.strftime("%Y-%m-%d"), 
        end_YEAR.strftime("%Y-%m-%d")))

Used extraction dates: 2015-10-22 --- 2015-11-07


In [262]:
start_YEAR, end_YEAR = start_YEAR.strftime("%Y-%m-%d"), end_YEAR.strftime("%Y-%m-%d")

In [263]:
df_disaster_start_end = df_time_interval(disaster_df_quotes, start_YEAR, end_YEAR,date_attr='date')

In [316]:
df_disaster_start_end.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2015-10-30-057627,"It kept me up all night, the wind. The windows...",Jorge Rodriguez,"['Q27574431', 'Q3295308', 'Q6278572', 'Q627857...",2015-10-30 06:09:05,1,"[['Jorge Rodriguez', '0.5058'], ['None', '0.49...",['http://www.kesq.com/news/strong-winds-blast-...,E
1,2015-10-29-104475,The Baker Polito Administration is fully devot...,Peter Lorenz,['Q1336282'],2015-10-29 10:49:41,3,"[['Peter Lorenz', '0.6971'], ['None', '0.2916'...",['http://insideclimatenews.org/news/29102015/m...,E
2,2015-11-06-095945,"The chance of rain really ramps up around 9 a.m.,",Ryan Chamberlain,"['Q17153294', 'Q47434958']",2015-11-06 09:58:19,1,"[['Ryan Chamberlain', '0.5887'], ['None', '0.4...",['http://pnj.com/story/news/military/2015/11/0...,E
6,2015-11-06-019594,"During the summit, we will review the progress...",Narendra Modi,['Q1058'],2015-11-06 16:00:22,16,"[['Narendra Modi', '0.8165'], ['None', '0.1482...",['http://www.newindianexpress.com/nation/PM-to...,E


In [280]:
tags_pos_list = disaster_YEAR_pos.tags.values.tolist()

In [281]:
tags_pos_list

['(?=.*\\b([mM]exic(o|ans?)|[tT]ehuantepec|[cC]oasts?|[jJ]alisco)\\b)(?=.*\\b([hH]urricanes?|[sS]torms?)\\b)',
 '(?=.*\\b([mM]exic(o|ans?)|[tT]ehuantepec|[cC]oasts?|[jJ]alisco)\\b)(?=.*\\b([fF]lood(waters?|s|ed|ing)?|[rR]ain(ed|s|fall)?|[lL]andslides?)\\b)',
 '\\b(Patricia)\\b',
 '\\b([cC]ategory 5)\\b',
 '\\b(NOAA)\\b',
 '\\b(National Hurricane Center|NHC)\\b',
 '\\b(Mexican (Red Cross|Army|Navy|Federal Police))\\b']

In [282]:
# Return regex pattern
regex_pattern_pos = r'|'.join(tags_pos_list)

In [283]:
filtered_for_pos = extract_quotes(df_disaster_start_end, regex_pattern_pos).sort_values(by='date')

In [284]:
len(filtered_for_pos)

223

In [309]:
filtered_for_pos.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
18400,2015-10-22-106707,"The mother of Sean Smith, one of the four Amer...",None,[],2015-10-22 08:06:00,1,"[['None', '0.6205'], ['Ben Carson', '0.2369'],...",['http://feeds.washingtonpost.com/c/34656/f/63...,E
8169,2015-10-22-108390,The problem for us is that species that don't ...,None,[],2015-10-22 10:08:08,1,"[['None', '0.7835'], ['Paul Pape', '0.2165']]",['http://www.texasobserver.org/wildfires-likel...,E
3829,2015-10-22-009987,As we celebrate 41 magical years at the Texas ...,Albert Hall,"['Q1387383', 'Q4710404']",2015-10-22 11:56:13,1,"[['Albert Hall', '0.5299'], ['None', '0.4415']...",['http://www.kbtx.com/home/headlines/Texas-Ren...,E
11232,2015-10-22-123312,This year's faculty art exhibit will feature a...,Brian Robinson,"['Q16533484', 'Q17027609', 'Q18157438', 'Q2696...",2015-10-22 14:24:58,1,"[['Brian Robinson', '0.7679'], ['None', '0.230...",['http://www.t-g.com/story/2243104.html'],E


In [311]:
extract_quotes(filtered_for_pos, r'\b(some_pattern)\b', complement=False).quotation.values

array([], dtype=object)

In [312]:
extract_quotes(filtered_for_pos, r'some_pattern', field='urls', complement=False).quotation.values

array([], dtype=object)

In [204]:
tag_neg_list = disaster_YEAR_neg.tags.values.tolist()


regex_pattern_neg = r'|'.join(tag_neg_list)

In [205]:
filtered_for_pos_then_neg = extract_quotes(filtered_for_pos, regex_pattern_neg, complement=True)

In [206]:
len(filtered_for_pos_then_neg)

5022

In [313]:
filtered_for_pos_then_neg.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
1506,2017-08-25-025209,facing first serious crises with Hurricane Har...,Matt Drudge,['Q1567226'],2017-08-25,5,"[['Matt Drudge', '0.5346'], ['President Donald...",['http://euronews.com/2017/08/25/hurricane-har...,E
30922,2017-08-25-055076,"If granted by the president, this request will...",None,[],2017-08-25,6,"[['None', '0.7802'], ['Greg Abbott', '0.2198']]",['http://keranews.org/post/harvey-makes-landfa...,E
10464,2017-08-25-104517,"Texas is about to get hit by a major hurricane,",Brock Long,['Q29964476'],2017-08-25,277,"[['Brock Long', '0.7948'], ['None', '0.1923'],...",['http://abcnews.go.com/US/hurricane-harvey-br...,E
9031,2017-08-25-077677,Keep on top of Hurricane Harvey. Don't make sa...,Chuck Grassley,['Q529294'],2017-08-25,105,"[['Chuck Grassley', '0.7189'], ['None', '0.200...",['https://www.vanguardngr.com/2017/08/disaster...,E


In [ ]:
apply_extra = filtered_for_pos_then_neg # or filtered_for_pos

In [299]:
filtered_extra = storm_tags_2015_extra()

In [300]:
len(filtered_extra)

119

In [314]:
filtered_extra.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
8169,2015-10-22-108390,The problem for us is that species that don't ...,None,[],2015-10-22 10:08:08,1,"[['None', '0.7835'], ['Paul Pape', '0.2165']]",['http://www.texasobserver.org/wildfires-likel...,E
3829,2015-10-22-009987,As we celebrate 41 magical years at the Texas ...,Albert Hall,"['Q1387383', 'Q4710404']",2015-10-22 11:56:13,1,"[['Albert Hall', '0.5299'], ['None', '0.4415']...",['http://www.kbtx.com/home/headlines/Texas-Ren...,E
18575,2015-10-22-100402,The ANUENUE Network is a testimony to the supe...,Governor David Ige,['Q16189318'],2015-10-22 14:57:08,1,"[['Governor David Ige', '0.6821'], ['James Jen...",['http://www.bigislandvideonews.com/2015/10/22...,E
17500,2015-10-22-034269,Hurricane Sandy changed forever how we think a...,None,[],2015-10-22 16:39:55,2,"[['None', '0.5751'], ['Curt Johnson', '0.4249']]",['http://yosemite.epa.gov/opa/admpress.nsf/0/F...,E


In [305]:
YEAR, disaster_type

(2015, 'storm')

In [306]:
output_df = filtered_extra # Or filtered_for_pos_then_neg or filtered_for_pos

In [315]:
output_df.head(4)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
8169,2015-10-22-108390,The problem for us is that species that don't ...,None,[],2015-10-22 10:08:08,1,"[['None', '0.7835'], ['Paul Pape', '0.2165']]",['http://www.texasobserver.org/wildfires-likel...,E
3829,2015-10-22-009987,As we celebrate 41 magical years at the Texas ...,Albert Hall,"['Q1387383', 'Q4710404']",2015-10-22 11:56:13,1,"[['Albert Hall', '0.5299'], ['None', '0.4415']...",['http://www.kbtx.com/home/headlines/Texas-Ren...,E
18575,2015-10-22-100402,The ANUENUE Network is a testimony to the supe...,Governor David Ige,['Q16189318'],2015-10-22 14:57:08,1,"[['Governor David Ige', '0.6821'], ['James Jen...",['http://www.bigislandvideonews.com/2015/10/22...,E
17500,2015-10-22-034269,Hurricane Sandy changed forever how we think a...,None,[],2015-10-22 16:39:55,2,"[['None', '0.5751'], ['Curt Johnson', '0.4249']]",['http://yosemite.epa.gov/opa/admpress.nsf/0/F...,E


In [308]:
write_df_to_disk(output_df, disaster_type, YEAR, additional_text='filtered')